<a href="https://colab.research.google.com/github/Prisci723/InteligenciaArtificial1/blob/main/Laboratorio%205%20Redes%20neuronales/Lab5_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Información sobre el dataset

https://archive.ics.uci.edu/dataset/31/covertype

El siguiente dataset trata sobre el tipo de covertura de un bosque, cuenta con los datos cartográficos de bosques, recolectados por el gobierno de Estados Unidos, cuenta con las siguientes variables:
* X1	Elevación en metros
* X2	Aspecto
* X3	Pendiente
* X4	Distancia_horizontal_a_hidrología
* X5	Distancia_vertical_a_hidrología
* X6	Distancia_horizontal_a_las_carreteras
* X7	sombreado_9am
* X8	Hillshade_mediodía
* X9	sombreado_3pm
* X10	Distancia_horizontal_a_puntos_de_fuego
* X11	Designación de área silvestre (clasificación)
* X12	Designación de área silvestre Clasificación
* X13	Designación de área silvestre Clasificación
* X14	Designación de área silvestre Clasificación
* [X15 - X55]	Designación del tipo de suelo 40 columnas de clasificación


Target:

y: Designación del tipo de cubierta forestal (1-7)


Con la siguiente inforamción se procede a la limpieza del dataset utilizando pandas

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv("/content/gdrive/MyDrive/Machine learning/Datasets/covtype.csv", sep=",")
print(dataframe)

        2596   51   3  258    0   510  221  232  148  6279  ...  0.34  0.35  \
0       2590   56   2  212   -6   390  220  235  151  6225  ...     0     0   
1       2804  139   9  268   65  3180  234  238  135  6121  ...     0     0   
2       2785  155  18  242  118  3090  238  238  122  6211  ...     0     0   
3       2595   45   2  153   -1   391  220  234  150  6172  ...     0     0   
4       2579  132   6  300  -15    67  230  237  140  6031  ...     0     0   
...      ...  ...  ..  ...  ...   ...  ...  ...  ...   ...  ...   ...   ...   
581006  2396  153  20   85   17   108  240  237  118   837  ...     0     0   
581007  2391  152  19   67   12    95  240  237  119   845  ...     0     0   
581008  2386  159  17   60    7    90  236  241  130   854  ...     0     0   
581009  2384  170  15   60    5    90  230  245  143   864  ...     0     0   
581010  2383  165  13   60    4    67  231  244  141   875  ...     0     0   

        0.36  0.37  0.38  0.39  0.40  0.41  0.42  5

**Comprobados si existen valores nulos**

In [ ]:
print(dataframe.isnull().any())

2596    False
51      False
3       False
258     False
0       False
510     False
221     False
232     False
148     False
6279    False
1       False
0.1     False
0.2     False
0.3     False
0.4     False
0.5     False
0.6     False
0.7     False
0.8     False
0.9     False
0.10    False
0.11    False
0.12    False
0.13    False
0.14    False
0.15    False
0.16    False
0.17    False
0.18    False
0.19    False
0.20    False
0.21    False
0.22    False
0.23    False
0.24    False
0.25    False
0.26    False
0.27    False
0.28    False
0.29    False
0.30    False
0.31    False
1.1     False
0.32    False
0.33    False
0.34    False
0.35    False
0.36    False
0.37    False
0.38    False
0.39    False
0.40    False
0.41    False
0.42    False
5       False
dtype: bool


**Comprobados si existen valores no numéricos dentro del dataset**

In [ ]:
# Verificar si hay valores no numéricos en las columnas excepto la fecha[columnas_numericas]
non_numeric_columns = dataframe.apply(lambda x: pd.to_numeric(x, errors='coerce')).isnull().any()
print("Valores no numéricos en las columnas:")
print(non_numeric_columns)

Valores no numéricos en las columnas:
2596    False
51      False
3       False
258     False
0       False
510     False
221     False
232     False
148     False
6279    False
1       False
0.1     False
0.2     False
0.3     False
0.4     False
0.5     False
0.6     False
0.7     False
0.8     False
0.9     False
0.10    False
0.11    False
0.12    False
0.13    False
0.14    False
0.15    False
0.16    False
0.17    False
0.18    False
0.19    False
0.20    False
0.21    False
0.22    False
0.23    False
0.24    False
0.25    False
0.26    False
0.27    False
0.28    False
0.29    False
0.30    False
0.31    False
1.1     False
0.32    False
0.33    False
0.34    False
0.35    False
0.36    False
0.37    False
0.38    False
0.39    False
0.40    False
0.41    False
0.42    False
5       False
dtype: bool


In [ ]:
print(dataframe['5'].unique())

[5 2 1 7 3 6 4]


Como el dataset está limpio y no posee inconsistencias, el modelo comienza a ser aplicado

# Aplicación del modelo

Se importan las librerías necesarias para ser utilizadas por la red neuronal

In [3]:

import os

import numpy as np

from matplotlib import pyplot

from scipy import optimize

%matplotlib inline

In [4]:
import sys

sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks/machine learning/datasets')
sys.path.insert(1, '/content/gdrive/MyDrive/Colab Notebooks/machine learning/03 redes neuronales')


El dataset es importado

In [5]:
import shutil

shutil.copy('/content/gdrive/MyDrive/Machine learning/Datasets/covtype.csv', '/content/data')

'/content/data'

In [6]:

data = np.loadtxt('/content/gdrive/MyDrive/Machine learning/Datasets/covtype.csv', delimiter=',')

# X y Y son extraidos del dataset
X, y = data[:, 0:-1], data[:,-1]

#Los valores del target son comprobados que sean enteros y también garantiza de que sea un array de una sola dimensión
y = np.array([int(e) for e in y])
print(y.shape)
y = np.squeeze(y)

#Los valores de y cambian para que comienzen en 0
y[y == 1] = 0
y[y == 2] = 1
y[y == 3] = 2
y[y == 4] = 3
y[y == 5] = 4
y[y == 6] = 5
y[y == 7] = 6

#Obtenemos el 80% de los datos y separamos el entrenamiento de la prueba
test = X.shape[0]*0.8
test = round(test)

Xp= X[test:,:]
X = X[:test,:]

yp = y[test:]
y = y[:test]

#Se imprime el tamaño de los arrays separados
print(X.shape)
print(y.shape)
print(Xp.shape)
print(yp.shape)

print(y)
print(yp)

(581012,)
(464810, 54)
(464810,)
(116202, 54)
(116202,)
[4 4 1 ... 0 1 1]
[1 1 1 ... 2 2 2]


**Normalización de los datos**

Los datos son normalizados ya que se encuentran en distintas escalas, y también a que se ocupen menos recursos al momento de su optimización.

In [7]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [8]:
X, mu, sigma = featureNormalize(X)

print(X)
print('Media calculada:', mu)
print('Desviación estandar calculada:', sigma)
print(X)

[[-1.22267023 -0.90636509 -1.4244481  ... -0.16428962 -0.14043752
  -0.10991019]
 [-1.24422982 -0.86153978 -1.55741491 ... -0.16428962 -0.14043752
  -0.10991019]
 [-0.47527104 -0.11743951 -0.62664724 ... -0.16428962 -0.14043752
  -0.10991019]
 ...
 [ 0.13199081  1.60385266 -0.49368044 ... -0.16428962 -0.14043752
  -0.10991019]
 [ 0.13558407  1.57695747 -0.62664724 ... -0.16428962 -0.14043752
  -0.10991019]
 [ 0.14995713  1.43351645 -0.22774682 ... -0.16428962 -0.14043752
  -0.10991019]]
Media calculada: [2.93626717e+03 1.52099686e+02 1.37128095e+01 2.62642127e+02
 4.50795723e+01 2.50147360e+03 2.12845328e+02 2.23118352e+02
 1.41790030e+02 2.05627125e+03 5.51618941e-01 1.62862245e-03
 3.67218864e-01 7.95335729e-02 6.52094404e-03 1.05806674e-02
 5.44523569e-03 1.63916439e-02 3.43581248e-03 1.41455649e-02
 2.25898754e-04 3.85103591e-04 2.46767496e-03 5.79355005e-02
 1.77577935e-02 6.44801102e-02 2.64559713e-02 8.08932682e-04
 6.45425012e-06 5.44523569e-03 6.87162497e-03 3.93494116e-03
 6.

En esta parte se establecerá las capas y la cantidad de entradas y como estarán configuradas para aplicar el modelo.

In [9]:
# Configurando parametros necesario
input_layer_size  = 54  # 54 features en total
hidden_layer_size = 30   # 30 perceptrones en la capa oculta
num_labels = 7          # La salida son 7 tipos del 0 al 6

pesos = {}
pesos['Theta1'] = np.random.rand(30, 55)
pesos['Theta2'] = np.random.rand(7, 31)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(1650,)
(217,)
(1867,)


La función de activación es establecida, también el gradiente de la sigmoide

In [10]:
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

La siguiente función es la más importante del modelo, ya que en esta parte es que se calculará el costo y el gradiente para ser aplicados en el modelo y así poder obtener la salida de la red neuronal.

Comienza con el cálculo de capa por capa.

In [11]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_= 0.0):

    #Theta1 y Theta2 cambian de dimensión para poder se utilizados
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    #Fin de la redimensionamiento
    #Los pesos son calculados para cada capa
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)

    a3 = sigmoid(a2.dot(Theta2.T))


    y_matrix = y.reshape(-1)

    y_matrix = np.eye(num_labels)[y_matrix]


    temp1 = Theta1
    temp2 = Theta2

    # La regularización es añadida a la función de costo que BCE

    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))

    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    # Backpropogation

    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # La regularización es añadida al descenso de gradiente

    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]

    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]


    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    return J, grad

Demostración e inicialización de los parametros que vayan a ser aplicados dentro de la red neuronal

In [12]:
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
print('El costo debe esta cercano a               : 0.287629')

Costo en parametros (cargado de ex4weights): 47.894055 
El costo debe esta cercano a               : 0.287629


In [13]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):

    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [15]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Los parametros ininciales son puestos en una sola dimensión
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


El modelo es aplicado al dataset, con los pasos previamente definidos y se obtienen los valores que permitirán hacer predicciones

In [16]:
#lambda se define al igual que el numero de veces que se optimizará
lambda_ = 0.01
max_iterations = 200

# La funcion de costo es aplicada al modelo
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

#funcion que permite minimizar la funcion de costo
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='L-BFGS-B',  # Cambio de metodo 'L-BFGS-B', para poder definir las iteraciones
                        options={'maxiter': max_iterations})
# se obtienen los valores
nn_params = res.x

# Theta1  y Theta2 obtienen su forma inicial
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

# Prueba del modelo
Ahora que tenemos los valores de theta, es que se pueden hacer las predicciones

In [17]:
def predict(Theta1, Theta2, X):


    m = X.shape[0]
    num_labels = Theta2.shape[0]

    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

**Predicción para los datos del entrenamiento**

In [18]:
pred = predict(Theta1, Theta2, X[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[1 1 1 ... 0 0 0]
Training Set Accuracy: 81.393257


Los datos de prueba de son normalizados para que estén en una misma escala

In [19]:
def  testNormalize(X, mu, sigma):
    X_norm = X.copy()
    X_norm = (X - mu) / sigma
    m, n = X_norm.shape
    return X_norm

In [20]:
X_test_norm = testNormalize(Xp, mu, sigma)

**Predicción para los datos de prueba**

In [21]:
pred = predict(Theta1, Theta2, X_test_norm[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == yp[:]) * 100))


[0 0 0 ... 2 2 2]
Training Set Accuracy: 60.571247
